In [15]:
from vivarium import Artifact
import pandas as pd

In [16]:
old_artifact = Artifact('/mnt/team/simulation_science/pub/models/vivarium_gates_nutrition_optimization_child/artifacts/model_15.0/pakistan.hdf')
new_artifact = Artifact('/mnt/team/simulation_science/pub/models/vivarium_gates_nutrition_optimization_child/artifacts/mean/pakistan.hdf')


In [17]:
keys_list =new_artifact.keys
keys_list

['metadata.keyspace', 'metadata.locations', 'population.location']

In [18]:
for key in old_artifact.keys:
    if key in list(['metadata.keyspace','metadata.locations','population.location']):
        continue
    data = old_artifact.load(key)
    old_artifact.remove(key)
    if isinstance(data, pd.DataFrame) and 'draw_0' in data.columns and data['draw_0'].dtype == float:
        data['mean_draw'] = data.filter(like='draw_').mean(axis=1)
        data = data.drop(columns=data.filter(like='draw_').columns)
        data = data.rename(columns={'mean_draw': 'draw_0'})
        if "exposure" in key:
            df_wide = data.unstack()
            s_original_param_sums = df_wide.sum(axis=1)

            # Normalize dataframe
            df_normalized_wide = df_wide.div(s_original_param_sums, axis=0)
            df_normalized = df_normalized_wide.stack()
            data = df_normalized


    new_artifact.write(key, data)